### Tensorflow with datetime

In [ ]:
import tensorflow as tf
import numpy as np
from datetime import datetime

In [41]:
x = tf.Variable(3, name="x")
y = tf.Variable(5, name="y")
f = x*x*7 + y + 2

In [42]:
with tf.Session() as sess:
    sess.run(x.initializer)
    sess.run(y.initializer)
    result = sess.run(f)
    print(result)

70


In [43]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

70


In [44]:
np.c_[np.ones((3,1)), [[1,2],[3,4],[5,6]]]

array([[1., 1., 2.],
       [1., 3., 4.],
       [1., 5., 6.]])

In [45]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
print(now)

20200205225703


In [46]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
print(now)

20200205225705


In [47]:
root_logdir = 'tf_logs'
logdir = "{}/run-{}/".format(root_logdir, now)
print(logdir)

tf_logs/run-20200205225705/


### Using autodiff and saver

In [ ]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [81]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [107]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [109]:
n_epochs = 1000
learning_rate = 0.01

In [110]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)
#替换上面的gradients and training_op
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9)
#training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            save_path = saver.save(sess, "/tmp/my_model_" + str(epoch) + ".ckpt")
            print("Epoch", epoch, "MSE =", mse.eval()) 
        sess.run(training_op)

    best_theta = theta.eval()
    save_path = saver.save(sess, "tmp/my_model_final.ckpt")

Epoch 0 MSE = 9.440211
Epoch 100 MSE = 0.8241427
Epoch 200 MSE = 0.65427864
Epoch 300 MSE = 0.6236339
Epoch 400 MSE = 0.60196555
Epoch 500 MSE = 0.5853587
Epoch 600 MSE = 0.57254153
Epoch 700 MSE = 0.56259704
Epoch 800 MSE = 0.55484134
Epoch 900 MSE = 0.5487615


In [111]:
best_theta

array([[ 2.0685525 ],
       [ 0.9286099 ],
       [ 0.16376483],
       [-0.40307665],
       [ 0.39767882],
       [ 0.01058714],
       [-0.0452447 ],
       [-0.4787963 ],
       [-0.45857945]], dtype=float32)

### Mini-batch Gradient Descent

In [154]:
#reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [155]:
learning_rate = 0.01
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [156]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [157]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size = batch_size)  # not shown 0:m-1有放回的gen batch_size个数
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        #n_batches = 207
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
                #file_writer.add_summary(summary_str)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [158]:
file_writer.close()

In [159]:
best_theta

array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.1173173 ],
       [-0.22739051],
       [ 0.3113402 ],
       [ 0.00353193],
       [-0.01126994],
       [-0.91643935],
       [-0.8795008 ]], dtype=float32)